<a href="https://colab.research.google.com/github/adeveloperdiary/DeepLearning_MiniProjects/blob/master/Predict_Basketball_Scores/Using_Deep_Learning_to_Predict_Basketball_Scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [266]:
import numpy
from platform import python_version
import pandas as pd
import tensorflow as tf
import numpy as np

print(f'python version:{python_version()}')
print(f'numpy version: {numpy.version.version}')
print(f'pandas version: {pd.__version__}')
print(f'tensorflow version: {tf.__version__}')

python version:3.6.9
numpy version: 1.18.5
pandas version: 1.1.2
tensorflow version: 2.3.0


In [267]:
from google.colab import drive
drive.mount('/content/drive')
ROOT_DIR='/content/drive/My Drive/Colab Notebooks/datasets/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [268]:
games_raw=pd.read_csv(f'{ROOT_DIR}/Games.csv',names=['date','defending-team','d-score','challenging-team','c-score'],parse_dates=['date'],header=None)
teams_raw=pd.read_csv(f'{ROOT_DIR}/Teams.csv',names=['conference','team-name'],header=None)

In [269]:
games_raw

,date,defending-team,d-score,challenging-team,c-score
0,2015-11-13,Hawaii,87,Montana St.,76
1,2015-11-13,Eastern Mich.,70,Vermont,50
2,2015-11-13,Columbia,107,Kean,62
3,2015-11-13,La.-Monroe,88,McMurry,43
4,2015-11-13,Yale,70,Fairfield,57
...,...,...,...,...,...
23488,2019-03-21,Hampton,81,St. Francis (B&#039;klyn),72
23489,2019-03-21,La.-Monroe,87,Kent State,77
23490,2019-03-21,CSU Fullerton,58,CSU Bakersfield,66
23491,2019-03-22,Southern Utah,80,Drake,73


In [270]:
teams_raw

,conference,team-name
0,America East,Vermont
1,America East,Stony Brook
2,America East,UMBC
3,America East,Hartford
4,America East,Albany
...,...,...
348,Western Athletic,California Baptist
349,Western Athletic,Cal State Bakersfield
350,Western Athletic,Seattle
351,Western Athletic,UMKC


In [271]:
teams_raw.set_index('team-name',drop=True, inplace=True)
teams_dict=teams_raw.to_dict(orient="index")

In [272]:
games_raw=games_raw.dropna()

In [274]:
games_raw['winner'] = np.where(games_raw['d-score']>games_raw['c-score'], games_raw['defending-team'], games_raw['challenging-team'])
games_raw['home-court'] = np.where(games_raw['winner']==games_raw['defending-team'], 1, 0)
games_raw['looser'] = np.where(games_raw['d-score']< games_raw['c-score'], games_raw['defending-team'], games_raw['challenging-team'])
games_raw.drop(['defending-team','challenging-team'],inplace=True,axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [275]:
def find_winner_conf(row):  
  name=row['winner']  

  if 'St.' in name and name not in teams_dict:
    name=name.replace('St.','State')
  elif 'Mich.' in name and name not in teams_dict:
    name=name.replace('Mich.','Michigan')
  elif 'Ala.' in name and name not in teams_dict:
    name=name.replace('Ala.','Alabama')
  elif 'U.' in name and name not in teams_dict:
    name=name.replace('U.','University')  
  elif 'Ill.' in name and name not in teams_dict:
    name=name.replace('Ill.','Illinois')  
  elif 'Ky.' in name and name not in teams_dict:
    name=name.replace('Ky.','Kentucky')  
  elif '&amp;' in name and name not in teams_dict:
    name=name.replace('&amp;','&')  
  elif 'Ga.' in name and name not in teams_dict:
    name=name.replace('Ga.','Georgia')  
  elif 'Caro.' in name and name not in teams_dict:
    name=name.replace('Caro.','Carolina')  
  elif '&#039;' in name and name not in teams_dict:
    name=name.replace('&#039;',"'")  
  

  if name in teams_dict:
    return teams_dict[name]['conference']
  else:    
    return ''

def find_loosing_conf(row):  
  name=row['looser']  

  if 'St.' in name and name not in teams_dict:
    name=name.replace('St.','State')
  elif 'Mich.' in name and name not in teams_dict:
    name=name.replace('Mich.','Michigan')
  elif 'Ala.' in name and name not in teams_dict:
    name=name.replace('Ala.','Alabama')
  elif 'U.' in name and name not in teams_dict:
    name=name.replace('U.','University')  
  elif 'Ill.' in name and name not in teams_dict:
    name=name.replace('Ill.','Illinois')  
  elif 'Ky.' in name and name not in teams_dict:
    name=name.replace('Ky.','Kentucky')  
  elif '&amp;' in name and name not in teams_dict:
    name=name.replace('&amp;','&')  
  elif 'Ga.' in name and name not in teams_dict:
    name=name.replace('Ga.','Georgia')  
  elif 'Caro.' in name and name not in teams_dict:
    name=name.replace('Caro.','Carolina')  
  elif '&#039;' in name and name not in teams_dict:
    name=name.replace('&#039;',"'")  
  

  if name in teams_dict:
    return teams_dict[name]['conference']
  else:    
    return ''

In [276]:
def update_team_names(old_name,new_name):
  games_raw.loc[games_raw['winner']==old_name,'winner']=new_name
  games_raw.loc[games_raw['looser']==old_name,'looser']=new_name


name_change=[('La.-Monroe','Louisiana-Monroe'),
             ('St. Francis (B&#039;klyn)','St Francis (BKN)'),
             ('CSU Fullerton','Cal State Fullerton'),
             ('Md.-East. Shore','Maryland-Eastern Shore'),
             ('St. John&#039;s (NY)','St John\'s'),
             ('St Johns','St John\'s'),
             ('UNC Wilmington','North Carolina-Wilmington'),
             ('Fla. Atlantic','Florida Atlantic'),
             ('Col. of Charleston','Charleston'),
             ('Loyola Maryland','Loyola (MD)'),
             ('UConn','Connecticut'),
             ('St. Bonaventure','St Bonaventure'),
             ('Central Conn. St.','Central Connecticut State'),
             ('Southern California','California'),
             ('Neb. Omaha','Nebraska-Omaha'),
             ('Louisiana','Louisiana Tech'),
             ('St. Peter&#039;s','St Peter\'s'),
             ('St Peters','St Peter\'s'),
             ('Saint Joseph&#039;s','Saint Joseph\'s (PA)'),
             ('South Fla.','South Florida'),
             ('SIU Edwardsville','SIU-Edwardsville'),
             ('St. Mary&#039;s (CA)','Saint Mary\'s'),
             ('East Tenn. St.','East Tennessee State'),
             ('Eastern Wash.','Eastern Washington'),
             ('UT Martin','Tennessee-Martin'),
             ('Southern Miss.','Southern Miss'),
             ('McNeese ','McNeese State'),
             ('Northern Colo.','Northern Colorado'),
             ('UT Arlington ','Texas-Arlington'),
             ('Southeastern La.','Southeastern Louisiana'),
             ('Albany (NY)','Albany'),
             ('S.C. Upstate','USC Upstate'),
             ('Bakersfield','Cal State Bakersfield'),
             ('Detroit','Detroit Mercy'),
             ('St. Francis (PA)','St Francis (PA)'),
             ('Mass.-Lowell','Massachusetts-Lowell'),
             ('N.C. A&amp;T','North Carolina A&T'),
             ('Northern Ariz.','Northern Arizona'),
             ('Stephen F. Austin','Stephen F Austin'),
             ('Charleston Southern','Charleston So.'),
             ('Mt. St. Mary&#039;s','Mount St Mary\'s'),
             ('LSU Shreveport','LSU'),
             ('Bryant University','Bryant'),
             ('S. Carolina St.','South Carolina State'),
             ('Geo. Washington','George Washington'),
             ('N&#039;western St.','Northwestern State'),
             ('Loyola Chicago','Loyola (IL)'),
             ('Lamar University','Lamar'),
             ('Fort Wayne','Purdue Fort Wayne'),
             ('Mississippi','Mississippi State'),
             ('Cal Baptist','California Baptist'),
             ('Southern Univ.','Southern University'),
             ('UNCG','UNC Greensboro'),
             ('Miami (Ohio)','Miami (OH)'),
             ('UIC','Illinois-Chicago'),
             ('Fairleigh D&#039;son','Fairleigh Dickinson'),
             ('UC Santa Barb.','UC Santa Barbara'),
             ('LMU','Loyola Marymount'),
             ('UMES','Maryland-Eastern Shore'),
             ('Cent. Conn. St.','Central Connecticut State'),
             ('Mich. St. ','Michigan State'),
             ('Loyola (Md.)','Loyola (MD)'),
             ('Ohio St. ','Ohio State'),
             ('St. Francis (Pa.)','St Francis (PA)'),
             ('Miami (Fla.)','Miami (FL)'),
             ('UNCW','North Carolina-Wilmington'),
             ('CSU Bakersfield','Cal State Bakersfield'),
             ('Omaha','Nebraska-Omaha'),
             ('Seattle U','Seattle'),
             ('Little Rock','Arkansas-Little Rock'),
             ('St. Mary&#039;s (Cal.)','Saint Mary\'s'),
             ('Texas A&amp;M-C.C.','Texas A&M'),
             ('CSUN','Cal State Northridge'),             
             ('SIUE','SIU-Edwardsville'),
             ('SFA','Stephen F Austin'),
             ('NC State','North Carolina State'),
             ('Army West Point','Army'),
             ('St. John&#039;s ','St John\'s'),
             ('Saint Joseph\'s','Saint Joseph\'s (PA)'),
             ('Prairie View','Prairie View A&M'),
             ('Central Ark.','Central Arkansas'),
             ('Mississippi Val.','Mississippi Valley State'),
             ('Ark.-Pine Bluff','Arkansas-Pine Bluff'),
             ('Southeast Mo. St.','Southeast Missouri State'),
             ('Middle Tenn.','Middle Tennessee'),
             ('Grambling','Grambling State'),
             ('UTRGV','Texas Rio Grande Valley'),
             ('Charleston So.','Charleston Southern'),          
             ('Bryant','Bryant University'),
             ('American','American University'),
             ('VMI','Virginia Military'),
             ('St. Francis (NY)','St Francis (BKN)'),
             ('N.C. Central','North Carolina Central'),
             ('IPFW','Purdue Fort Wayne'),
             ('FGCU','Florida Gulf Coast'),
             ('UALR','Arkansas-Little Rock'),
             ('UNI','Northern Iowa'),
             ('VCU','Virginia Commonwealth'),
             ('FIU','Florida International'),
             ('Penn','Pennsylvania')
             ]

for old_name,new_name in name_change:
  update_team_names(old_name,new_name)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [277]:
games_raw['winning-team-conf']=games_raw.apply(find_winner_conf, axis=1)
games_raw['loosing-team-conf']=games_raw.apply(find_loosing_conf, axis=1)

print(games_raw[games_raw['winning-team-conf']=='']['winner'].unique().size)
games_raw[games_raw['loosing-team-conf']=='']['looser'].unique().size

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


34


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


575

In [278]:
games_raw['year']=games_raw['date'].dt.year.astype('Int64')
games_raw['month']=games_raw['date'].dt.month.astype('Int64')
games_raw['day']=games_raw['date'].dt.day.astype('Int64')
games_raw.drop('date', axis=1, inplace=True)

games_raw['d-score']>games_raw['c-score']
games_raw


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,d-score,c-score,winner,home-court,looser,winning-team-conf,loosing-team-conf,year,month,day
0,87,76,Hawaii,1,Montana St.,Big West,Big Sky,2015,11,13
1,70,50,Eastern Mich.,1,Vermont,Mid-American,America East,2015,11,13
2,107,62,Columbia,1,Kean,Ivy League,,2015,11,13
3,88,43,Louisiana-Monroe,1,McMurry,Sun Belt,,2015,11,13
4,70,57,Yale,1,Fairfield,Ivy League,Metro Atlantic Athletic,2015,11,13
...,...,...,...,...,...,...,...,...,...,...
23488,81,72,Hampton,1,St Francis (BKN),Big South,Northeast,2019,3,21
23489,87,77,Louisiana-Monroe,1,Kent State,Sun Belt,Mid-American,2019,3,21
23490,58,66,Cal State Bakersfield,0,Cal State Fullerton,Western Athletic,Big West,2019,3,21
23491,80,73,Southern Utah,1,Drake,Big Sky,Missouri Valley,2019,3,22


In [280]:
games_raw['score-diff']=(games_raw['d-score']-games_raw['c-score']).abs()
games_raw

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,d-score,c-score,winner,home-court,looser,winning-team-conf,loosing-team-conf,year,month,day,score-diff
0,87,76,Hawaii,1,Montana St.,Big West,Big Sky,2015,11,13,11
1,70,50,Eastern Mich.,1,Vermont,Mid-American,America East,2015,11,13,20
2,107,62,Columbia,1,Kean,Ivy League,,2015,11,13,45
3,88,43,Louisiana-Monroe,1,McMurry,Sun Belt,,2015,11,13,45
4,70,57,Yale,1,Fairfield,Ivy League,Metro Atlantic Athletic,2015,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...
23488,81,72,Hampton,1,St Francis (BKN),Big South,Northeast,2019,3,21,9
23489,87,77,Louisiana-Monroe,1,Kent State,Sun Belt,Mid-American,2019,3,21,10
23490,58,66,Cal State Bakersfield,0,Cal State Fullerton,Western Athletic,Big West,2019,3,21,8
23491,80,73,Southern Utah,1,Drake,Big Sky,Missouri Valley,2019,3,22,7


In [296]:
team_names=pd.concat([pd.Series(games_raw['winner'].unique()), pd.Series(games_raw['looser'].unique())], axis=0).unique()
team_names=np.random.permutation(team_names)

team_names_dict2int={name:i for i,name in enumerate(team_names)}
team_names_int2dict={team_names_dict2int[k]:k for k in team_names_dict2int.keys()}
print(team_names_dict2int)
print(team_names_int2dict)

{'Michigan': 0, 'CCNY': 1, 'Norfolk State': 2, 'Chicago State': 3, 'Olivet Nazarene': 4, 'Utah St.': 5, 'East Tex. Baptist': 6, 'Davidson': 7, 'Centenary (N.J.)': 8, 'Rochester (MI)': 9, 'Cin Clermont': 10, 'St. Mary&#039;s (Md.)': 11, 'Wofford': 12, 'Marquette': 13, 'E. Texas Bapt.': 14, 'Butler': 15, 'Angelo St.': 16, 'Toccoa Falls': 17, 'Siena Heights': 18, 'North Carolina St.': 19, 'Drake': 20, 'Southern Utah': 21, 'Hardin-Simmons': 22, 'Rice': 23, 'Union (NY)': 24, 'Chapman': 25, 'Florida A&amp;M': 26, 'Trinity Int&#039;l': 27, 'Cal St. L.A.': 28, 'Tabor': 29, 'Idaho St.': 30, 'BYU': 31, 'Molloy': 32, 'Delaware': 33, 'Fort Valley St.': 34, 'Cal St. East Bay': 35, 'LeMoyne-Owen': 36, 'Niagara': 37, 'Emerson': 38, 'Virginia Commonwealth': 39, 'Northern Ill.': 40, 'Caltech': 41, 'Bowie St.': 42, 'Rutgers-Newark': 43, 'Montreat': 44, 'Washington Col.': 45, "Saint Joseph's (PA)": 46, 'McNeese St.': 47, 'Louisiana Col.': 48, 'CSU Dom. Hills': 49, 'Tusculum': 50, 'Missouri Valley': 51, '

In [299]:
games_raw['winner_num']=games_raw['winner'].apply(lambda x: team_names_dict2int[x])
games_raw['looser_num']=games_raw['looser'].apply(lambda x: team_names_dict2int[x])
games_raw

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,d-score,c-score,winner,home-court,looser,winning-team-conf,loosing-team-conf,year,month,day,score-diff,winner_num,looser_num
0,87,76,Hawaii,1,Montana St.,Big West,Big Sky,2015,11,13,11,641,686
1,70,50,Eastern Mich.,1,Vermont,Mid-American,America East,2015,11,13,20,501,523
2,107,62,Columbia,1,Kean,Ivy League,,2015,11,13,45,806,563
3,88,43,Louisiana-Monroe,1,McMurry,Sun Belt,,2015,11,13,45,746,284
4,70,57,Yale,1,Fairfield,Ivy League,Metro Atlantic Athletic,2015,11,13,13,144,895
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23488,81,72,Hampton,1,St Francis (BKN),Big South,Northeast,2019,3,21,9,135,585
23489,87,77,Louisiana-Monroe,1,Kent State,Sun Belt,Mid-American,2019,3,21,10,746,449
23490,58,66,Cal State Bakersfield,0,Cal State Fullerton,Western Athletic,Big West,2019,3,21,8,336,219
23491,80,73,Southern Utah,1,Drake,Big Sky,Missouri Valley,2019,3,22,7,21,20


In [300]:
conf_names=pd.concat([pd.Series(games_raw['winning-team-conf'].unique()), pd.Series(games_raw['loosing-team-conf'].unique())], axis=0).unique()
conf_names=np.random.permutation(conf_names)

conf_names_dict2int={name:i for i,name in enumerate(conf_names)}
conf_names_int2dict={conf_names_dict2int[k]:k for k in conf_names_dict2int.keys()}
print(conf_names_dict2int)
print(conf_names_int2dict)

{'West Coast': 0, 'America East': 1, 'Atlantic 10': 2, 'Big Ten': 3, 'American Athletic Conference': 4, 'Big 12': 5, 'Pac-12': 6, 'Big South': 7, 'Mountain West': 8, '': 9, 'Southwestern Athletic': 10, 'Western Athletic': 11, 'Atlantic Coast': 12, 'Northeast': 13, 'Colonial Athletic': 14, 'Big East': 15, 'Southern': 16, 'Summit League': 17, 'Mid-Eastern': 18, 'Big West': 19, 'Horizon': 20, 'Mid-American': 21, 'Missouri Valley': 22, 'Ivy League': 23, 'Conference USA': 24, 'Southeastern': 25, 'Ohio Valley': 26, 'Atlantic Sun': 27, 'Metro Atlantic Athletic': 28, 'Patriot League': 29, 'Big Sky': 30, 'Southland': 31, 'Sun Belt': 32}
{0: 'West Coast', 1: 'America East', 2: 'Atlantic 10', 3: 'Big Ten', 4: 'American Athletic Conference', 5: 'Big 12', 6: 'Pac-12', 7: 'Big South', 8: 'Mountain West', 9: '', 10: 'Southwestern Athletic', 11: 'Western Athletic', 12: 'Atlantic Coast', 13: 'Northeast', 14: 'Colonial Athletic', 15: 'Big East', 16: 'Southern', 17: 'Summit League', 18: 'Mid-Eastern', 19

In [302]:
games_raw['winning-team-conf_num']=games_raw['winning-team-conf'].apply(lambda x: conf_names_dict2int[x])
games_raw['loosing-team-conf-num']=games_raw['loosing-team-conf'].apply(lambda x: conf_names_dict2int[x])

games_raw.drop(['winner','looser','winning-team-conf','loosing-team-conf'],inplace=True, axis=1)

games_raw

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

,d-score,c-score,home-court,year,month,day,score-diff,winner_num,looser_num,winning-team-conf_num,loosing-team-conf-num
0,87,76,1,2015,11,13,11,641,686,19,30
1,70,50,1,2015,11,13,20,501,523,21,1
2,107,62,1,2015,11,13,45,806,563,23,9
3,88,43,1,2015,11,13,45,746,284,32,9
4,70,57,1,2015,11,13,13,144,895,23,28
...,...,...,...,...,...,...,...,...,...,...,...
23488,81,72,1,2019,3,21,9,135,585,7,13
23489,87,77,1,2019,3,21,10,746,449,32,21
23490,58,66,0,2019,3,21,8,336,219,11,19
23491,80,73,1,2019,3,22,7,21,20,30,22


In [305]:
y=np.where(games_raw['d-score']>games_raw['c-score'], games_raw['d-score'],games_raw['c-score'])
games_raw.drop(['d-score','c-score'],inplace=True, axis=1)
games_raw

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,home-court,year,month,day,score-diff,winner_num,looser_num,winning-team-conf_num,loosing-team-conf-num
0,1,2015,11,13,11,641,686,19,30
1,1,2015,11,13,20,501,523,21,1
2,1,2015,11,13,45,806,563,23,9
3,1,2015,11,13,45,746,284,32,9
4,1,2015,11,13,13,144,895,23,28
...,...,...,...,...,...,...,...,...,...
23488,1,2019,3,21,9,135,585,7,13
23489,1,2019,3,21,10,746,449,32,21
23490,0,2019,3,21,8,336,219,11,19
23491,1,2019,3,22,7,21,20,30,22


In [326]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse=False)
cat_data=games_raw.drop(['home-court','year','month','day','score-diff'],axis=1)
one_hot_data=one_hot_encoder.fit_transform(cat_data.values)
print(one_hot_data.shape)
num_data=games_raw.drop(['winner_num','looser_num','winning-team-conf_num','loosing-team-conf-num'],axis=1).values

X=np.concatenate([num_data,one_hot_data],axis=1)
print(X.shape)

np.savetxt(f'{ROOT_DIR}/Games-Calculated.csv',X,delimiter=',')


(23492, 1487)
(23492, 1492)
